In [ ]:
!pip install open_clip_torch
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from PIL import Image
import open_clip
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')
model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv('gdrive/My Drive/capstone/our_foods.csv', header=None, dtype = str)
df2 = df.iloc[:,1]
print(df2[0])
dflist=df2.values.tolist()
dflist_str = [str(x) for x in dflist]
##dfnum=df2.to_numpy(dtype=str)
print(dflist_str[100000])

# image = preprocess(Image.open('gdrive/My Drive/capstone/foodtest.png')).unsqueeze(0)

Hummus, commercial
CASHEW DRINK


In [ ]:
start = time.time()
##text = open_clip.tokenize(["asian chicken", "rice", "asian vegetables", "asian beef"])
text = open_clip.tokenize(dflist_str)
end = time.time()
print((end-start),"sec")

40.195841789245605 sec


In [ ]:
i=129000
with torch.no_grad(), torch.cuda.amp.autocast():
    # image_features = model.encode_image(image)
    while(i<240000):
      start = time.time()
      text_features = model.encode_text(text[i-1000:i])
      end = time.time()
      print(str(i), (end-start), "sec")
      torch.save(text_features, "gdrive/My Drive/capstone/text_feat"+str(i-1000)+"-"+str(i-1)+".pt")
      i=i+1000

129000 78.50542116165161 sec
130000 76.93301296234131 sec
131000 77.122145652771 sec
132000 78.60228228569031 sec
133000 78.20496940612793 sec
134000 77.04437446594238 sec
135000 77.00552487373352 sec
136000 78.05194306373596 sec
137000 77.02637386322021 sec
138000 77.00634360313416 sec
139000 78.02476167678833 sec
140000 77.07257175445557 sec
141000 76.99918699264526 sec
142000 78.1781165599823 sec
143000 76.97826147079468 sec
144000 76.93497633934021 sec
145000 78.01660180091858 sec
146000 76.93391466140747 sec
147000 76.91385006904602 sec
148000 77.8500645160675 sec
149000 76.82996654510498 sec
150000 76.87386870384216 sec
151000 76.85774636268616 sec
152000 77.62781167030334 sec
153000 76.81297755241394 sec
154000 76.78652477264404 sec
155000 77.64901852607727 sec
156000 76.947274684906 sec
157000 76.82084965705872 sec
158000 77.59799075126648 sec
159000 76.79889369010925 sec
160000 76.96937584877014 sec
161000 77.61267495155334 sec
162000 76.87560200691223 sec
163000 77.0321321487

In [ ]:
from sys import getsizeof
print(getsizeof(text[0]))
print(getsizeof(text))
print(text[100000])
print(type(text))
print(len(text))
print(text[:3])

88
88
tensor([49406, 39531,  3979, 49407,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
<class 'torch.Tensor'>
377603
tensor([[49406, 31785,   267,  6287, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

In [ ]:
with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    start = time.time()
    text_features = model.encode_text(text[:1000])
    end = time.time()
    print((end-start),"sec")
    image_features /= image_features.norm(dim=-1, keepdim=True)
    start = time.time()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    end = time.time()
    print((end-start),"sec")
    start = time.time()
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    end = time.time()
    print((end-start),"sec")
    start = time.time()
    text_probs2 = (100.0 * image_features @ text_features.T)
    end = time.time()
    print((end-start),"sec")
print("Label probs:", text_probs, text_probs2)  # prints: [[1., 0., 0.]]

658.9008224010468 sec
0.0008275508880615234 sec
0.0006759166717529297 sec
0.0013964176177978516 sec
Label probs: tensor([[1.0105e-08, 7.6316e-10, 1.2304e-07, 2.6504e-06, 4.8442e-07, 2.6956e-09,
         7.4752e-08, 8.2607e-09, 1.6984e-09, 7.0508e-08, 1.1134e-06, 1.4963e-08,
         2.9201e-08, 1.5210e-08, 4.3707e-11, 3.0893e-07, 2.1165e-07, 1.9206e-05,
         6.8909e-07, 2.0313e-09, 3.5311e-12, 1.9215e-09, 1.1138e-08, 1.4884e-09,
         1.1447e-08, 4.2461e-08, 1.0696e-08, 3.5902e-07, 4.0283e-09, 3.2326e-08,
         4.3027e-02, 1.0166e-01, 1.0276e-07, 1.8337e-07, 1.7010e-08, 1.1472e-07,
         7.9552e-09, 6.3739e-08, 1.0225e-07, 1.2042e-05, 2.3553e-01, 9.3781e-05,
         6.0358e-05, 4.6815e-07, 2.1491e-08, 2.2291e-08, 1.0960e-08, 3.6771e-08,
         1.6686e-07, 5.4592e-07, 7.2668e-09, 5.1108e-08, 6.2167e-10, 4.4090e-09,
         4.3907e-07, 1.9193e-06, 6.7793e-09, 7.8480e-11, 5.0254e-08, 2.2293e-09,
         1.1548e-01, 3.9483e-07, 1.1723e-08, 1.3739e-07, 1.2822e-09, 3.2340e-

In [ ]:
torch.save(text_features, 'gdrive/My Drive/capstone/text_features0-1000.pt')

In [ ]:
text_features2=torch.load('gdrive/My Drive/capstone/text_features0-1000.pt')
print(getsizeof(text_features2))
print(getsizeof(text_features2[0]))
print(text_features2)
print(text_features2[0])
print(len(text_features2))
print(len(text_features2[0]))
# print(text[:3])
print(text_features2[0][0])
print((text_features2[0][0]).element_size())

88
88
tensor([[ 0.0055,  0.0271, -0.0400,  ...,  0.0134, -0.0272,  0.0060],
        [ 0.0294, -0.0069, -0.0289,  ...,  0.0238,  0.0677,  0.0348],
        [-0.0427, -0.0185, -0.0231,  ..., -0.0043,  0.0376,  0.0155],
        ...,
        [-0.0155,  0.0059,  0.0073,  ...,  0.0142,  0.0119, -0.0087],
        [ 0.0294,  0.0379, -0.0097,  ...,  0.0084,  0.0027,  0.0685],
        [ 0.0097, -0.0143, -0.0198,  ..., -0.0319,  0.0030,  0.0420]])
tensor([ 0.0055,  0.0271, -0.0400,  ...,  0.0134, -0.0272,  0.0060])
1000
1024
tensor(0.0055)
4


In [ ]:
textcat = torch.cat((text_features2, text_features2))
print(textcat)
print(len(textcat))
print(textcat[1000:1003])

tensor([[ 0.0055,  0.0271, -0.0400,  ...,  0.0134, -0.0272,  0.0060],
        [ 0.0294, -0.0069, -0.0289,  ...,  0.0238,  0.0677,  0.0348],
        [-0.0427, -0.0185, -0.0231,  ..., -0.0043,  0.0376,  0.0155],
        ...,
        [-0.0155,  0.0059,  0.0073,  ...,  0.0142,  0.0119, -0.0087],
        [ 0.0294,  0.0379, -0.0097,  ...,  0.0084,  0.0027,  0.0685],
        [ 0.0097, -0.0143, -0.0198,  ..., -0.0319,  0.0030,  0.0420]])
2000
tensor([[ 0.0055,  0.0271, -0.0400,  ...,  0.0134, -0.0272,  0.0060],
        [ 0.0294, -0.0069, -0.0289,  ...,  0.0238,  0.0677,  0.0348],
        [-0.0427, -0.0185, -0.0231,  ..., -0.0043,  0.0376,  0.0155]])


In [ ]:
print(getsizeof(text_features))
print(getsizeof(text_features[0]))
print(text_features)
print(text_features[0])
print(len(text_features))
print(len(text_features[0]))
# print(text[:3])
print(text_features[0][0])
print((text_features[0][0]).element_size())

88
88
tensor([[ 0.0923,  0.4566, -0.6739,  ...,  0.2255, -0.4590,  0.1007],
        [ 0.4215, -0.0993, -0.4148,  ...,  0.3422,  0.9717,  0.4999],
        [-0.7189, -0.3107, -0.3887,  ..., -0.0729,  0.6334,  0.2616],
        ...,
        [-0.1608, -0.4754, -0.1260,  ..., -0.4127,  0.0459,  0.1824],
        [ 0.2812, -0.4964,  0.3803,  ..., -0.8039,  0.1779, -0.0657],
        [-0.4230,  0.0710,  0.1605,  ..., -0.1230, -0.7868,  0.5485]])
tensor([ 0.0923,  0.4566, -0.6739,  ...,  0.2255, -0.4590,  0.1007])
10
1024
tensor(0.0923)
4


In [ ]:
with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    text_probs2 = (100.0 * image_features @ text_features.T)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
print("Label probs:", text_probs, text_probs2)  # prints: [[1., 0., 0.]]